In [1]:
import torch
import json
import time
import statistics
import numpy as np
import pickle as pkl

from tqdm import tqdm


from transformers import AutoTokenizer
from transformers.models.bloom.configuration_bloom import BloomConfig
from pruning.pruned_bloom import PrunedBloomForCausalLM
from node_attribution.utils import count_params
from rouge_score import rouge_scorer

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(f"bigscience/bloomz-1b1")

In [3]:
data = pkl.load(open("44_human_filtered_conv_pairs.pkl", "rb"))
cali_data = data[:22]
val_data = data[22:]

In [4]:
def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.pad_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.pad_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

In [5]:
def calc_p(data):
    perplexity_sum = 0
    for pair in tqdm(data):
        perplexity = score(sentence=pair, model=pruned_model, tokenizer=tokenizer)
        perplexity_sum += perplexity
    
    p = perplexity_sum / len(val_data)
    
    return p

In [8]:
weights_path = "pruned_40percent_560m_bloom.pt"
state_dict_shapes_path = "pruned_40percent_560m_bloom_state_dict_shapes.pkl"

bloom_config = BloomConfig(
    vocab_size=250880,
    hidden_size=1024,
    n_layer=24,
    n_head=16,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
    use_cache=True,
    bos_token_id=1,
    eos_token_id=2,
    apply_residual_connection_post_layernorm=False,
    hidden_dropout=0.0,
    attention_dropout=0.0,
    pretraining_tp=1,  # TP rank used when training with megatron
    slow_but_exact=False,
    attention_softmax_in_fp32=True,
    bias_dropout_fusion=True,
    masked_softmax_fusion=True,
    offset_alibi=100,
    pad_token_id=3,
    seq_length=2048,
    skip_bias_add=True,
    skip_bias_add_qkv=False,
    unk_token_id=0,
    
)

In [9]:
pruned_model = PrunedBloomForCausalLM(bloom_config, state_dict_shapes_path)

In [10]:
pruned_model.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [11]:
pruned_percent = 1.0 - (count_params(pruned_model)[-1] / 559214592)
print(pruned_percent)

0.1600794440642922


In [12]:
print(calc_p(cali_data))
print(calc_p(val_data))

 36%|████████████████████████████████████████████████████▋                                                                                            | 8/22 [01:15<02:12,  9.45s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/qm/qmg6cwns11x93fg6_77cntd40000gn/T/ipykernel_54539/4175705424.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/qm/qmg6cwns11x93fg6_77cntd40000gn/T/ipykernel_54539/4175705424.py'                 │
│                                                                                                  │
│ /var/folders/qm/qmg6cwns11x93fg6_77cntd40000gn/T/ipykernel_54539/3088197817.py:4 in calc_p       │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/qm/qmg6cwns11x93fg6_77cntd40000gn/T/ipykernel_54539/3088197817.py'                 │
│                                                                                                  │
│ /var/folders/qm/qmg6cwns11x93fg6_77cntd40000gn/T/ipykernel_54539/792537457.py:8 in score         │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/qm/qmg6cwns11x93fg6_77cntd40000gn/T/ipykernel_54539/792537457.py'                  │
│                                                                                                  │
│ /opt/homebrew/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in _call_impl         │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /opt/homebrew/lib/python3.10/site-packages/transformers/models/bloom/modeling_bloom.py:900 in    │
│ forward                                                                                          │
│                                                                                                  │
│    897 │   │                                                                                     │
│    898 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│    899 │   │                                                                                     │
│ ❱  900 │   │   transformer_outputs = self.transformer(                                           │
│    901 │   │   │   input_ids,                                                                    │
│    902 │   │   │   past_key_values=past_key_values,                                              │
│    903 │   │   │   attention_mask=attention_mask,                                                │
│                                                                                                  │
│ /opt/homebrew/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in _call_impl         │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.     

In [ ]:
num_trials = 20

In [ ]:
line = "Hello, I am a social bot! "
inputs = tokenizer(line, return_tensors="pt")
pruned_times = []

for i in range(num_trials):
    start = time.time()
    outputs = pruned_model.generate(
        input_ids=inputs["input_ids"], 
        max_new_tokens=100, 
        do_sample=True, 
        top_k=50, 
        top_p=0.95,
    )
    end = time.time()
    pruned_times.append(end - start)
    print(f"inference time: {end - start}")
    
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [ ]:
mean_pruned = np.mean(pruned_times)
std_pruned = np.std(pruned_times)
mode = statistics.median(pruned_times)
print(mean_pruned)
print(std_pruned)
print(mode)

In [14]:
# while True:
#     line = input("You:")
#     inputs = tokenizer(line, return_tensors="pt")
#     outputs = pruned_model.generate(
#         input_ids=inputs["input_ids"], 
#         max_new_tokens=20, 
#         do_sample=True, 
#         top_k=50, 
#         top_p=0.95,
#     )
#     print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [14]:
# full_line = "Person: My favorite movie is the The Day After Tomorrow\nSocialBot: Oh, interesting, I am not familiar with that movie! Can you tell me more about it?"
# prompt_line = "Person: My favorite movie is The Day After Tomorrow\nSocialBot: "
# completion = full_line.split(prompt_line)[-1]
# inputs = tokenizer(prompt_line, return_tensors="pt")

# #for i in range(num_trials):
# outputs = pruned_model.generate(
#     input_ids=inputs["input_ids"], 
#     max_new_tokens=25, 
#     do_sample=True, 
#     top_k=50, 
#     top_p=0.95,
# )
# out_seq = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# out_seq = out_seq[0].split("Person: My favorite movie is The Day After Tomorrow\nSocialBot: ")[-1]
# r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
# rouge_scores = r_scorer.score(completion, out_seq)

In [15]:
# from transformers import AutoTokenizer, BloomForCausalLM

In [16]:
# model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

In [17]:
# inputs = tokenizer("Person: My favorite movie is The Day After Tomorrow\nSocialBot: ", return_tensors="pt")
# start = time.time()
# outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=25, do_sample=True, top_k=50, top_p=0.95)
# end = time.time()
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# print(f"inference time: {end - start}")